In [47]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
%matplotlib inline
import hvplot.pandas

In [15]:
load_dotenv()



True

In [18]:
alpaca_api_key = os.getenv("ALPACA_API_KEY_ID")
print("alpaca_api_key   ",type(alpaca_api_key))
# YOUR CODE HERE
alpaca_secret_key=os.getenv("SECRET_KEY")
print("alpaca_secret_key",type(alpaca_secret_key))
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
%matplotlib inline
import hvplot.pandas
# Create the Alpaca tradeapi.REST object
# YOUR CODE HERE
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

alpaca_api_key    <class 'str'>
alpaca_secret_key <class 'str'>


In [64]:
from alpaca_trade_api.rest import REST, TimeFrame

AAPL_df=alpaca.get_bars("AAPL", TimeFrame.Day, "2021-01-02", "2022-04-07", adjustment='raw').df
#df_portfolio.dropna()
#AAPL_df.head(10)
#AAPL_df
#AAPL_df["close"].plot()
#AAPL_df=pd.DataFrame(close,index=pd.DatetimeIndex)

AAPL_df=AAPL_df.drop(columns=["open","high","low","volume","trade_count","vwap"])
AAPL_df

,close
timestamp,
2021-01-04 05:00:00+00:00,129.41
2021-01-05 05:00:00+00:00,131.01
2021-01-06 05:00:00+00:00,126.60
2021-01-07 05:00:00+00:00,130.92
2021-01-08 05:00:00+00:00,132.05
...,...
2022-04-01 04:00:00+00:00,174.31
2022-04-04 04:00:00+00:00,178.44
2022-04-05 04:00:00+00:00,175.06


In [65]:
import hvplot.pandas
AAPL_df.hvplot.line()

:Curve   [timestamp]   (close)

In [66]:
AAPL_df["trade_type"]=np.nan

In [67]:
previous_price=0

In [68]:
for index,row in AAPL_df.iterrows():
    if previous_price==0:
        AAPL_df.loc[index,"trade_type"]="buy"
        
    elif row["close"] < previous_price:
            AAPL_df.loc[index,"trade_type"]="buy"
    

    elif row["close"] > (1.005*previous_price):
            AAPL_df.loc[index,"trade_type"]="sell"
    else:
         AAPL_df.loc[index, "trade_type"] = "hold"
            
previous_price=row["close"]

if index == AAPL_df.index[-1]:
    AAPL_df.loc[index, "trade_type"] = "sell"

AAPL_df.head(15)
            
        

,close,trade_type
timestamp,,
2021-01-04 05:00:00+00:00,129.41,buy
2021-01-05 05:00:00+00:00,131.01,buy
2021-01-06 05:00:00+00:00,126.60,buy
2021-01-07 05:00:00+00:00,130.92,buy
2021-01-08 05:00:00+00:00,132.05,buy
2021-01-11 05:00:00+00:00,128.98,buy
2021-01-12 05:00:00+00:00,128.80,buy
2021-01-13 05:00:00+00:00,130.89,buy
2021-01-14 05:00:00+00:00,128.91,buy


In [69]:
AAPL_df["cost/proceeds"]=np.nan
share_size = 1
accumulated_shares = 0


In [70]:
for index, row in AAPL_df.iterrows():

    # buy if the previous_price is 0, in other words, buy on the first day
    if previous_price == 0:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size

    # buy if the current day's price is less than the previous day's price
    elif row["close"] < previous_price:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size
    elif row["close"] > (1.005*previous_price):
        AAPL_df.loc[index, "trade_type"] = "sell"

In [71]:
if index == AAPL_df.index[-1]:
    AAPL_df.loc[index, "trade_type"] = "sell"

    # calculate the proceeds by multiplying the last day's price by the accumulated shares
   # amd_df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares

In [79]:
#Initialize trade_type column to track buys and sells
AAPL_df["trade_type"] = np.nan

# Initialize a cost/proceeds column for recording trade metrics
AAPL_df["cost/proceeds"] = np.nan
AAPL_df["accumulated_shares"] = np.nan
# Initialize share size and accumulated shares
share_size = 1
accumulated_shares = 0

# Initialize variable to hold previous price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a trade at each iteration
for index, row in AAPL_df.iterrows():

    # buy if the previous_price is 0, in other words, buy on the first day
    if previous_price == 0:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
    # buy if the current day's price is less than the previous day's price
    elif row["close"] < 0.98*previous_price:
        AAPL_df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        AAPL_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
    # hold if the current day's price is greater than the previous day's price
    elif row["close"] > 1.02*previous_price:
        AAPL_df.loc[index, "trade_type"] = "sell"
        AAPL_df.loc[index, "cost/proceeds"] = (row["close"] * accumulated_shares)
        accumulated_shares=0
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
        
    # hold if the current day's price is equal to the previous day's price
    else:
        AAPL_df.loc[index, "trade_type"] = "hold"
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
    # update the previous_price to the current row's price
    previous_price = row["close"]

    # if the index is the last index of the DataFrame, sell
    if index == AAPL_df.index[-1]:
        AAPL_df.loc[index, "trade_type"] = "sell"

        # calculate the proceeds by multiplying the last day's price by the accumulated shares
        AAPL_df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares
        AAPL_df.loc[index, "accumulated_shares"] = accumulated_shares
# Review the DataFrame
AAPL_df


,close,trade_type,cost/proceeds,accumulated_shares
timestamp,,,,
2021-01-04 05:00:00+00:00,129.41,buy,-129.41,1.0
2021-01-05 05:00:00+00:00,131.01,hold,NaN,1.0
2021-01-06 05:00:00+00:00,126.60,buy,-126.60,2.0
2021-01-07 05:00:00+00:00,130.92,sell,261.84,0.0
2021-01-08 05:00:00+00:00,132.05,hold,NaN,0.0
...,...,...,...,...
2022-04-01 04:00:00+00:00,174.31,hold,NaN,0.0
2022-04-04 04:00:00+00:00,178.44,sell,0.00,0.0
2022-04-05 04:00:00+00:00,175.06,hold,NaN,0.0


In [80]:
total_profit_loss = round(AAPL_df["cost/proceeds"].sum(),2)
print(f"The total profit/loss of the trading strategy is ${total_profit_loss}.")

The total profit/loss of the trading strategy is $71.84.


In [76]:
invested_capital = 0

# Calculate the invested capital by adding the cost of all buy trades
for index, row in AAPL_df.iterrows():
    if row["trade_type"] == "buy":
        invested_capital = invested_capital + row["cost/proceeds"]


# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)

# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")

The trading algorithm resulted in a return on investment of 1.49%


In [60]:
signals_df = AAPL_df.loc[:,["close"]]

short_window = 50
long_window = 100

signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0
signals_df

,close,SMA50,SMA100,Signal
timestamp,,,,
2022-01-03 05:00:00+00:00,182.010,NaN,NaN,0.0
2022-01-04 05:00:00+00:00,179.700,NaN,NaN,0.0
2022-01-05 05:00:00+00:00,174.835,NaN,NaN,0.0
2022-01-06 05:00:00+00:00,172.000,NaN,NaN,0.0
2022-01-07 05:00:00+00:00,172.170,NaN,NaN,0.0
...,...,...,...,...
2022-04-01 04:00:00+00:00,174.310,167.0210,NaN,0.0
2022-04-04 04:00:00+00:00,178.440,167.3416,NaN,0.0
2022-04-05 04:00:00+00:00,175.060,167.6104,NaN,0.0


In [61]:
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)
signals_df['Entry/Exit'] = signals_df['Signal'].diff()


In [62]:
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [63]:
AAPL_bt_df=alpaca.get_bars("AAPL", TimeFrame.Day, "2016-01-02", "2017-04-07", adjustment='raw').df


AAPL_bt_df=AAPL_bt_df.drop(columns=["open","high","low","volume","trade_count","vwap"])
AAPL_bt_df

,close
timestamp,
2016-01-04 05:00:00+00:00,105.35
2016-01-05 05:00:00+00:00,102.71
2016-01-06 05:00:00+00:00,100.70
2016-01-07 05:00:00+00:00,96.45
2016-01-08 05:00:00+00:00,96.96
...,...
2017-04-03 04:00:00+00:00,143.70
2017-04-04 04:00:00+00:00,144.77
2017-04-05 04:00:00+00:00,144.02
